---
## Iniciamos el Modelo

In [2]:
# Importar librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity

Se inicia creando los dummies 

In [3]:
Modelo= pd.read_parquet("../DataSets/steam_games.parquet")
Modelo.head(3)

,publisher,genres,title,release_date,tags,specs,price,early_access,id,developer
0,Kotoshiro,Action,Lost Summoner Kitty,2018,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",['Single-player'],4.99,False,761140,Kotoshiro
1,Kotoshiro,Casual,Lost Summoner Kitty,2018,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",['Single-player'],4.99,False,761140,Kotoshiro
2,Kotoshiro,Indie,Lost Summoner Kitty,2018,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",['Single-player'],4.99,False,761140,Kotoshiro


In [4]:
#Creacíon de dummies
Modelo= pd.get_dummies(Modelo, columns=["genres"], prefix="")

Modelo= Modelo.groupby(["id","title"]).sum().reset_index()

Modelo.head(4)

C:\Users\1\AppData\Local\Temp\ipykernel_2756\1420793241.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  Modelo= Modelo.groupby(["id","title"]).sum().reset_index()


,id,title,release_date,price,early_access,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
0,10,Counter-Strike,2000,9.99,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,20,Team Fortress Classic,1999,4.99,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30,Day of Defeat,2003,4.99,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,40,Deathmatch Classic,2001,4.99,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
similitudes = cosine_similarity(Modelo.iloc[:,3:])

In [7]:
similitudes.shape

(27181, 27181)

In [8]:
print(similitudes)

[[1.         0.99520055 0.99520055 ... 0.99682034 0.99641099 0.99520055]
 [0.99520055 1.         1.         ... 0.98423879 0.98350768 1.        ]
 [0.99520055 1.         1.         ... 0.98423879 0.98350768 1.        ]
 ...
 [0.99682034 0.98423879 0.98423879 ... 1.         0.99985554 0.98423879]
 [0.99641099 0.98350768 0.98350768 ... 0.99985554 1.         0.98350768]
 [0.99520055 1.         1.         ... 0.98423879 0.98350768 1.        ]]


Antes de seguir, pulimos un poco más nuestro archivo

In [9]:
#Creamos uan función para ayudarnos 

def buscar_datos_repetidos(data):
    """
    Busca datos duplicados en un DataFrame.

    Parameters:
    - data: DataFrame con los datos.

    Returns:
    - DataFrame con los datos duplicados.
    """
    duplicados = data[data.duplicated()]
    return duplicados

datos_duplicados = buscar_datos_repetidos(Modelo)

# Muestra los datos duplicados (si los hay)
if not datos_duplicados.empty:
    print("Datos duplicados encontrados:")
    print(datos_duplicados)
else:
    print("No se encontraron datos duplicados.")


No se encontraron datos duplicados.


In [9]:
buscar_datos_repetidos(Modelo)

,id,title,release_date,price,early_access,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing


In [10]:
Modelo.isnull().sum()

id                            0
title                         0
release_date                  0
price                         0
early_access                  0
_Action                       0
_Adventure                    0
_Animation &amp; Modeling     0
_Audio Production             0
_Casual                       0
_Design &amp; Illustration    0
_Early Access                 0
_Education                    0
_Free to Play                 0
_Indie                        0
_Massively Multiplayer        0
_Photo Editing                0
_RPG                          0
_Racing                       0
_Simulation                   0
_Software Training            0
_Sports                       0
_Strategy                     0
_Utilities                    0
_Video Production             0
_Web Publishing               0
dtype: int64

Planteamos una función de recomendación

In [10]:

#La función aun será ajustada
def recomendacion_juego(id_juego, modelo_item, similitudes):
    """
    Recomienda juegos similares a un juego dado.

    Parameters:
    - id_juego: ID del juego para el cual se desean recomendaciones.
    - modelo_item: DataFrame que contiene información sobre los juegos.
    - similitudes: Matriz de similitud entre juegos.

    Returns:
    - Lista de nombres de juegos recomendados.
    """
    try:
        id_juego = int(id_juego)
    except ValueError:
        return "Error: El ID del juego debe ser un número entero."

    # Filtrar el juego e igualarlo a su ID
    juego_seleccionado = modelo_item[modelo_item['id'] == id_juego]

    # Devolver error en caso de vacío
    if juego_seleccionado.empty:
        return "Error: El juego con el ID especificado no existe en la base de datos."

    # Calcular la similitud del juego que se ingresa con otros juegos del dataframe
    similarity_scores = similitudes[Modelo[Modelo['id'] == id_juego].index[0]]

    # Obtener los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similarity_scores.argsort()[::-1][1:6]

    # Obtener los nombres de los juegos 5 recomendados
    juegos_recomendados = modelo_item.iloc[indices_juegos_similares]['title'].tolist()

    return juegos_recomendados

id_juego = 761140
recomendaciones = recomendacion_juego(id_juego, Modelo, similitudes)



In [11]:
recomendaciones

['Lost Summoner Kitty',
 'R.C. Bot Inc.',
 'A.I. Space Corps',
 'Wooden Battles',
 'Demolition Inc.']

Buscamos juegos para iniciar las pruebas 

In [12]:
Modelo[Modelo["title"]=="FINAL FANTASY VII"] #Mi juego favorito de la saga

,id,title,release_date,price,early_access,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
976,39140,FINAL FANTASY VII,2013,11.99,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [13]:
Modelo[Modelo["title"]=="Cat Quest"]

,id,title,release_date,price,early_access,_Action,_Adventure,_Animation &amp; Modeling,_Audio Production,_Casual,...,_Photo Editing,_RPG,_Racing,_Simulation,_Software Training,_Sports,_Strategy,_Utilities,_Video Production,_Web Publishing
20740,593280,Cat Quest,8068,51.96,0,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


Tomamos la id y aplicamos la función 

In [14]:
id_juego2=39140
recoemdaciones_2=recomendacion_juego(id_juego2,Modelo, similitudes)

In [15]:
id_juego3=593280
recmendaciones_3=recomendacion_juego(id_juego3, Modelo, similitudes)

In [16]:
recoemdaciones_2

['古剑奇谭(GuJian)',
 'FINAL FANTASY VIII',
 'Taikou Risshiden / 太閤立志伝',
 'Tokyo Xanadu eX+: S-Pom Treat Bundle',
 'Ruzar - The Life Stone']

In [17]:
recmendaciones_3

['Pathologic Classic HD',
 'Dungeon Souls',
 'Age of Barbarian Extended Cut',
 "Vairon's Wrath",
 'Victor Vran: Mötorhead Through The Ages']

## Planteamos el modelo para consumirse en la API

In [18]:
# Obtengo la cantidad de filas
cantidad_filas = len(Modelo)

# Selecciono la mitad superior
Modelo_final = Modelo.iloc[:cantidad_filas // 2]


In [19]:
Modelo_final.shape

(13590, 26)

In [20]:
similitudes_final = cosine_similarity(Modelo_final.iloc[:,3:])

In [21]:
similitudes_final

array([[1.        , 0.99520055, 0.99520055, ..., 0.96831776, 0.99447462,
        0.85063592],
       [0.99520055, 1.        , 1.        , ..., 0.95418529, 0.97996039,
        0.87863932],
       [0.99520055, 1.        , 1.        , ..., 0.95418529, 0.97996039,
        0.87863932],
       ...,
       [0.96831776, 0.95418529, 0.95418529, ..., 1.        , 0.9726164 ,
        0.79190404],
       [0.99447462, 0.97996039, 0.97996039, ..., 0.9726164 , 1.        ,
        0.81329549],
       [0.85063592, 0.87863932, 0.87863932, ..., 0.79190404, 0.81329549,
        1.        ]])

Exportamos nuestro archivo a Parquet

In [24]:
Modelo_final.to_parquet("../DatosML/ModeloFinal.parquet")

--- 
## Pruebas y ajustes 

In [24]:
modelo_final=pd.read_parquet("../DatosML/ModeloFinal.parquet")

In [25]:

def encontrar_juegos_similares(id_juego, modelo_final):
    """
    Encuentra juegos similares a un juego dado por su ID.

    Parameters:
    - id_juego: ID del juego para el cual se desean encontrar juegos similares.
    - modelo_render: DataFrame que contiene las características de los juegos.

    Returns:
    - Lista de nombres de juegos similares.
    """
    # Encuentra el índice del juego ingresado por ID
    juego_indice = modelo_final.index[modelo_final['id'] == id_juego].tolist()

    # Verifica si el juego con el ID especificado existe en la base de datos
    if not juego_indice:
        return f"El juego con el ID {id_juego} no existe en la base de datos."

    juego_indice = juego_indice[0]

    # Extrae las características del juego ingresado
    juego_caracteristicas = modelo_final.iloc[juego_indice, 3:].values.reshape(1, -1)

    # Calcula la similitud coseno entre el juego ingresado y todos los demás juegos
    similitudes_render = cosine_similarity(modelo_final.iloc[:, 3:], juego_caracteristicas)

    # Obtiene los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similitudes_render.argsort(axis=0)[::-1][1:6].flatten()[1:]

    # Obtiene los juegos más similares en función de los índices
    juegos_similares = modelo_final.iloc[indices_juegos_similares]['title'].tolist()

    return juegos_similares




In [26]:
encontrar_juegos_similares(39140, modelo_final)

['Ruzar - The Life Stone',
 'Kagura Douchuuki',
 'Faery - Legends of Avalon',
 'Overlord II']

In [27]:
def recommend_games(game_id: int, modelo_final, sample_size=2000, random_state=42):
    '''
    Esta función recomienda 5 juegos a partir del juego ingresado.

    Args:
        game_id (int): ID único del videojuego al cual se le harán las recomendaciones.
        modelo_final (DataFrame): DataFrame que contiene las características de los juegos.
        sample_size (int): Tamaño de la muestra para calcular la similitud de contenido.
        random_state (int): Semilla aleatoria para reproducibilidad.

    Returns:
        Dict: Diccionario con la lista de 5 juegos recomendados.
    '''
    # Verifica si el juego con game_id existe en modelo_render
    game = modelo_final[modelo_final['id'] == game_id]

    if game.empty:
        return {"message": f"El juego con ID '{game_id}' no existe en la base de datos."}

    # Obtiene el índice del juego dado
    idx = game.index[0]

    # Toma una muestra aleatoria del DataFrame modelo_render
    df_sample = modelo_final.sample(n=sample_size, random_state=random_state)

    # Calcula la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([modelo_final.iloc[idx, 3:]], df_sample.iloc[:, 3:])

    # Obtiene las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordena los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtiene los 5 juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:5]]

    # Lista de juegos similares (solo nombres)
    similar_game_names = df_sample['title'].iloc[similar_game_indices].tolist()

    return {"similar_games": similar_game_names}

In [28]:
recommend_games(39140, modelo_final, sample_size=2500, random_state=42)

{'similar_games': ['FINAL FANTASY VIII',
  'Wanderjahr',
  'STAR WARS™ Knights of the Old Republic™ II - The Sith Lords™',
  'Overlord II',
  'Silverfall: Earth Awakening']}

Podemos ver una mejora en esta función

In [30]:
recommend_games(593280, modelo_final, sample_size=2000, random_state=42)

{'message': "El juego con ID '593280' no existe en la base de datos."}

In [32]:
recommend_games(773690, modelo_final, sample_size=2000, random_state=42)

{'message': "El juego con ID '773690' no existe en la base de datos."}

## Función final

In [35]:
from fastapi import FastAPI, HTTPException
from typing import Dict, List
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

app = FastAPI()


In [37]:
def encontrar_juegos_similares(id_juego: int, modelo_final) -> List[str]:
    """
    Encuentra juegos similares a un juego dado por su ID.

    Parameters:
    - id_juego: ID del juego para el cual se desean encontrar juegos similares.
    - modelo_final: DataFrame que contiene las características de los juegos.

    Returns:
    - Lista de nombres de juegos similares.
    """
    # Encuentra el índice del juego ingresado por ID
    juego_indice = modelo_final.index[modelo_final['id'] == id_juego].tolist()

    # Verifica si el juego con el ID especificado existe en la base de datos
    if not juego_indice:
        raise HTTPException(status_code=404, detail=f"El juego con el ID {id_juego} no existe en la base de datos.")

    juego_indice = juego_indice[0]

    # Extrae las características del juego ingresado
    juego_caracteristicas = modelo_final.iloc[juego_indice, 3:].values.reshape(1, -1)

    # Calcula la similitud coseno entre el juego ingresado y todos los demás juegos
    similitudes_render = cosine_similarity(modelo_final.iloc[:, 3:], juego_caracteristicas)

    # Obtiene los índices de los juegos más similares (excluyendo el juego de entrada)
    indices_juegos_similares = similitudes_render.argsort(axis=0)[::-1][1:6].flatten()[1:]

    # Obtiene los juegos más similares en función de los índices
    juegos_similares = modelo_final.iloc[indices_juegos_similares]['title'].tolist()

    return juegos_similares

In [ ]:
recommend_games(39140, modelo_final)

---
## Funciones de prueba

In [31]:
import pandas as pd
from sklearn.metrics import precision_score

def recommend_games(game_id: int, modelo_final, sample_size=2000, random_state=42, num_recommendations=5):
    '''
    Esta función recomienda juegos a partir del juego ingresado.

    Args:
        game_id (int): ID único del videojuego al cual se le harán las recomendaciones.
        modelo_final (DataFrame): DataFrame que contiene las características de los juegos.
        sample_size (int): Tamaño de la muestra para calcular la similitud de contenido.
        random_state (int): Semilla aleatoria para reproducibilidad.
        num_recommendations (int): Número de juegos a recomendar.

    Returns:
        Dict: Diccionario con la lista de juegos recomendados.
    '''
    # Verifica si el juego con game_id existe en modelo_render
    game = Modelo_final[Modelo_final['id'] == game_id]

    if game.empty:
        return {"message": f"El juego con ID '{game_id}' no existe en la base de datos."}

    # Obtiene el índice del juego dado
    idx = game.index[0]

    # Toma una muestra aleatoria del DataFrame modelo_render
    df_sample = Modelo_final.sample(n=sample_size, random_state=random_state)

    # Calcula la similitud de contenido solo para el juego dado y la muestra
    sim_scores = cosine_similarity([Modelo_final.iloc[idx, 3:]], df_sample.iloc[:, 3:])

    # Obtiene las puntuaciones de similitud del juego dado con otros juegos
    sim_scores = sim_scores[0]

    # Ordena los juegos por similitud en orden descendente
    similar_games = [(i, sim_scores[i]) for i in range(len(sim_scores)) if i != idx]
    similar_games = sorted(similar_games, key=lambda x: x[1], reverse=True)

    # Obtiene los juegos más similares
    similar_game_indices = [i[0] for i in similar_games[:num_recommendations]]

    # Lista de juegos similares (solo nombres)
    similar_game_names = df_sample['title'].iloc[similar_game_indices].tolist()

    return {"similar_games": similar_game_names}

def recommend_games_by_genre(genre: str, modelo_final, num_recommendations=5):
    '''
    Esta función recomienda juegos a partir del género ingresado.

    Args:
        genre (str): Género de los juegos a recomendar.
        modelo_final (DataFrame): DataFrame que contiene las características de los juegos.
        num_recommendations (int): Número de juegos a recomendar.

    Returns:
        Dict: Diccionario con la lista de juegos recomendados.
    '''
    # Filtra los juegos por el género especificado
    genre_games = modelo_final[modelo_final['genre'] == genre]

    if genre_games.empty:
        return {"message": f"No hay juegos disponibles para el género '{genre}'."}

    # Toma una muestra aleatoria de los juegos del género
    df_sample = genre_games.sample(n=num_recommendations, random_state=42)

    # Lista de juegos recomendados (solo nombres)
    recommended_games = df_sample['title'].tolist()

    return {"recommended_games": recommended_games}

def evaluate_recommendations(predictions, true_labels):
    '''
    Evalúa la precisión de las recomendaciones. r

    Args:
        predictions (List): Lista de juegos recomendados.
        true_labels (List): Lista de juegos reales.

    Returns:
        float: Precisión del modelo.
    '''
    precision = precision_score(true_labels, predictions, average='micro')
    return precision


game_id: int, modelo_final, sample_size=2000, random_state=42, num_recommendations=5

In [32]:
recommend_games(773690,Modelo_final, sample_size=4000, random_state=42, num_recommendations=10)

{'message': "El juego con ID '773690' no existe en la base de datos."}